In [46]:
# %matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import os
import snappy
from snappy import Product, ProductIO, ProductUtils, WKTReader, HashMap, GPF

# For shapefiles
# import shapefile
# import pygeoif
path_to_sentinel_data = "./data/S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7.SAFE.zip"
product = ProductIO.readProduct(path_to_sentinel_data)

In [47]:
width = product.getSceneRasterWidth()
print("Width: {} px".format(width))
height = product.getSceneRasterHeight()
print("Height: {} px".format(height))
name = product.getName()
print("Name: {}".format(name))
band_names = product.getBandNames()
print("Band names: {}".format(", ".join(band_names)))

Width: 25485 px
Height: 16820 px
Name: S1A_IW_GRDH_1SDV_20240109T095534_20240109T095559_052028_06499B_56C7
Band names: Amplitude_VH, Intensity_VH, Amplitude_VV, Intensity_VV


In [48]:
def plotBand(product, band, vmin, vmax):
    band = product.getBand(band)
    w = band.getRasterWidth()
    h = band.getRasterHeight()
    print(w, h)
    band_data = np.zeros(w * h, np.float32)
    band.readPixels(0, 0, w, h, band_data)
    band_data.shape = h, w
    width = 12
    height = 12
    plt.figure(figsize=(width, height))
    imgplot = plt.imshow(band_data, cmap=plt.cm.binary, vmin=vmin, vmax=vmax)
    return imgplot

In [49]:
parameters = HashMap()
GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
parameters.put('orbitType', 'Sentinel Precise (Auto Download)')
parameters.put('polyDegree', '3')
parameters.put('continueOnFail', 'false')

apply_orbit_file = GPF.createProduct('Apply-Orbit-File', parameters, product)

Exception calling QC Rest API:  qc.sentinel1.eo.esa.int: Name or service not known

100% done.


In [50]:
width = apply_orbit_file.getSceneRasterWidth()
print("Width: {} px".format(width))
height = apply_orbit_file.getSceneRasterHeight()
print("Height: {} px".format(height))
band_names = apply_orbit_file.getBandNames()
print("Band names: {}".format(", ".join(band_names)))

Width: 25485 px
Height: 16820 px
Band names: Amplitude_VH, Intensity_VH, Amplitude_VV, Intensity_VV


In [51]:
parameters = HashMap()
parameters.put('outputSigmaBand', True)
parameters.put('sourceBands', 'Intensity_VV,Intensity_VH')
parameters.put('selectedPolarisations', "VV,VH")
# parameters.put('sourceBands', 'Intensity_VH')
# parameters.put('selectedPolarisations', "VH")
parameters.put('outputImageScaleInDb', False)

product_calibrated = GPF.createProduct("Calibration", parameters, apply_orbit_file)


100% done.


In [52]:
width = product_calibrated.getSceneRasterWidth()
print("Width: {} px".format(width))
height = product_calibrated.getSceneRasterHeight()
print("Height: {} px".format(height))
band_names = product_calibrated.getBandNames()
print("Band names: {}".format(", ".join(band_names)))

Width: 25485 px
Height: 16820 px
Band names: Sigma0_VV, Sigma0_VH


In [53]:
filterSizeY = '5'
filterSizeX = '5'
parameters = HashMap()
# parameters.put('sourceBands', 'Sigma0_VV')
parameters.put('filter', 'Lee')
parameters.put('filterSizeX', filterSizeX)
parameters.put('filterSizeY', filterSizeY)
parameters.put('dampingFactor', '2')
parameters.put('estimateENL', 'true')
parameters.put('enl', '1.0')
parameters.put('numLooksStr', '1')
parameters.put('targetWindowSizeStr', '3x3')
parameters.put('sigmaStr', '0.9')
parameters.put('anSize', '50')
speckle_filter = GPF.createProduct('Speckle-Filter', parameters,product_calibrated)


100% done.


In [54]:
width = speckle_filter.getSceneRasterWidth()
print("Width: {} px".format(width))
height = speckle_filter.getSceneRasterHeight()
print("Height: {} px".format(height))
band_names = speckle_filter.getBandNames()
print("Band names: {}".format(", ".join(band_names)))

Width: 25485 px
Height: 16820 px
Band names: Sigma0_VV, Sigma0_VH


In [55]:
parameters = HashMap()
parameters.put('demName', 'SRTM 3Sec')
parameters.put('pixelSpacingInMeter', 10.0)
# parameters.put('sourceBands', 'Sigma0_VV')
speckle_filter_tc = GPF.createProduct("Terrain-Correction", parameters, speckle_filter)


100% done.


In [56]:
width = speckle_filter_tc.getSceneRasterWidth()
print("Width: {} px".format(width))
height = speckle_filter_tc.getSceneRasterHeight()
print("Height: {} px".format(height))
band_names = speckle_filter_tc.getBandNames()
print("Band names: {}".format(", ".join(band_names)))

Width: 28464 px
Height: 22192 px
Band names: Sigma0_VV, Sigma0_VH


In [ ]:
ProductIO.writeProduct(speckle_filter_tc, "./data/final", 'BEAM-DIMAP')
os.path.exists("./data/final.dim")

True